In [1]:
fulldir="/Users/mahitha0x01/Documents/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/"
name="ppl13"

In [2]:
import cv2
from PIL import Image
import os
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import random
from math import log
from tqdm import tqdm
import numpy as np

In [3]:
def getImageMatrix(imageName):
    im = Image.open(imageName, mode="r", formats=None) 
    pix = im.load()
    color = 1
    if type(pix[0,0]) == int:
        color=0
    image_size = im.size 
    image_matrix = []
    for width in range(int(image_size[0])):
        row = []
        for height in range(int(image_size[1])):
                row.append((pix[width,height]))
        image_matrix.append(row)
    return image_matrix,image_size[0],image_size[1],color

In [4]:
image=fulldir+ name+"/"
ext='.bmp'
key = "abcdefghijklm"

In [5]:
def LogisticDecryption(imageName, key):
    N = 256
    key_list = [ord(x) for x in key]

    G = [key_list[0:4] ,key_list[4:8], key_list[8:12]]
    g = []
    R = 1
    for i in range(1,4):
        s = 0
        for j in range(1,5):
            s += G[i-1][j-1] * (10**(-j))
        g.append(s)
        R = (R*s) % 1
    
    L_x = (R + key_list[12]/256) % 1
    S_x = round(((g[0]+g[1]+g[2])*(10**4) + L_x *(10**4)) % 256)
    V1 = sum(key_list)
    V2 = key_list[0]
    for i in range(1,13):
        V2 = V2 ^ key_list[i]
    V = V2/V1

    L_y = (V+key_list[12]/256) % 1
    S_y = round((V+V2+L_y*10**4) % 256)
    C1_0 = S_x
    C2_0 = S_y
    
    C = round((L_x*L_y*10**4) % 256)
    I_prev = C
    I_prev_r = C
    I_prev_g = C
    I_prev_b = C
    I = C
    I_r = C
    I_g = C
    I_b = C
    x_prev = 4*(S_x)*(1-S_x)
    y_prev = 4*(L_x)*(1-S_y)
    x = x_prev
    y = y_prev
    imageMatrix, dw, dh, color = getImageMatrix(imageName)

    henonDecryptedImage = []
    for i in range(dw):
        row = []
        for j in range(dh):
            i_x=0
            while x <0.8 and x > 0.2 and i_x<=10:
                x = 4*x*(1-x)
                i_x=i_x+1
            i_y=0
            while y <0.8 and y > 0.2 and i_y<=10:
                y = 4*y*(1-y)
                i_y=i_y+1
            x_round = round((x*(10**4))%256)
            y_round = round((y*(10**4))%256)
            C1 = x_round ^ ((key_list[0]+x_round) % N) ^ ((C1_0 + key_list[1])%N)
            C2 = x_round ^ ((key_list[2]+y_round) % N) ^ ((C2_0 + key_list[3])%N) 
            if color:
                I_r = ((((key_list[4]+C1) % N) ^ ((key_list[5]+C2) % N) ^ ((I_prev_r + key_list[7]) % N) ^ imageMatrix[i][j][0]) + N-key_list[6])%N
                I_g = ((((key_list[4]+C1) % N) ^ ((key_list[5]+C2) % N) ^ ((I_prev_g + key_list[7]) % N) ^ imageMatrix[i][j][1]) + N-key_list[6])%N
                I_b = ((((key_list[4]+C1) % N) ^ ((key_list[5]+C2) % N) ^ ((I_prev_b + key_list[7]) % N) ^ imageMatrix[i][j][2]) + N-key_list[6])%N
                I_prev_r = imageMatrix[i][j][0]
                I_prev_g = imageMatrix[i][j][1]
                I_prev_b = imageMatrix[i][j][2]
                row.append((I_r,I_g,I_b))
                x = (x +  imageMatrix[i][j][0]/256 + key_list[8]/256 + key_list[9]/256) % 1
                y = (x +  imageMatrix[i][j][0]/256 + key_list[8]/256 + key_list[9]/256) % 1  
            else:
                I = ((((key_list[4]+C1) % N) ^ ((key_list[5]+C2) % N) ^ ((I_prev+key_list[7]) % N) ^ imageMatrix[i][j]) + N-key_list[6])%N
                I_prev = imageMatrix[i][j]
                row.append(I)
                x = (x +  imageMatrix[i][j]/256 + key_list[8]/256 + key_list[9]/256) % 1
                y = (x +  imageMatrix[i][j]/256 + key_list[8]/256 + key_list[9]/256) % 1
            for ki in range(12):
                key_list[ki] = (key_list[ki] + key_list[12]) % 256
                key_list[12] = key_list[12] ^ key_list[ki]
        henonDecryptedImage.append(row)
    if color:
        im = Image.new("RGB", (dw, dh))
    else: 
        im = Image.new("L", (dw, dh)) # L is for Black and white pixels
    pix = im.load()
    for x in range(dw):
        for y in range(dh):
            pix[x, y] = henonDecryptedImage[x][y]
    print(imageName[28:-4])
    im.save(image+"decryp"+ext, "BMP")

In [6]:
def DecryptionResult(i, orig):
    if i<10:
        imageName=fulldir+name+'/encrypted_frames/EncryptedResult00'+str(i)+'.bmp'
    elif i<100:
        imageName=fulldir+name+'/encrypted_frames/EncryptedResult0'+str(i)+'.bmp'
    else:
        imageName=fulldir+name+'/encrypted_frames/EncryptedResult'+str(i)+'.bmp'
    #im = cv2.cvtColor(np.array( Image.open(imageName)), cv2.COLOR_RGB2BGR)
    #print(type(im))
    #na = np.array(im)
    #orig = na.copy()
    na=cv2.imread(imageName)
    print(na.shape)
    color_pixels= np.where(np.all(na==[255, 0, 0],axis=2))
    #print(color_pixels)
    
    zipped = np.column_stack((color_pixels[0], color_pixels[1]))

    #print(zipped)
    squares=[]
    for [y,x] in zipped:
        if np.array_equal(na[y+1,x], np.array([255, 0, 0]))==True and np.array_equal(na[y, x+1], np.array([255, 0, 0]))==True:
            #find the width and height
            w=0
            while (x+1+w)<na.shape[1] and  np.array_equal(na[y,x+1+w], np.array([255, 0, 0])):
                w=w+1
            h=0
            while (y+1+h)<na.shape[0] and np.array_equal(na[y+1+h,x], np.array([255, 0, 0])):
                h=h+1
            if w>1 and h>1:
                squares.append([x,y,w,h])
    print(squares)
    #print("frame: ", cnt)
    sum=0
    for [x,y,w,h] in squares:
        #print(a,b,c,d)
        face=na[y+1:y+h,x+1:x+w]
        if len(face)==0:
            continue
        filename=image+"decryp"+ext
        cv2.imwrite(filename, face)
        LogisticDecryption(filename, key)
        filename=image+"decryp"+ext
        DecryptResult=cv2.imread(filename)
        na[y+1:y+h,x+1:x+w]=DecryptResult
        na[y,x]=np.mean(np.array([na[y-1, x], na[y, x-1]]),axis=0) 
        na[y+1:y+h,x]=np.mean(np.array([na[y+1:y+h, x-1], na[y+1:y+h, x+1]]),axis=0) 
        na[y+h, x]=np.mean(np.array([na[y+h+1, x], na[y+h, x-1]]),axis=0)
        na[y+h, x+1:x+w]=np.mean(np.array([na[y+h-1, x+1:x+w], na[y+h+1, x+1:x+w]]),axis=0) 
        na[y+h, x+w]=np.mean(np.array([na[y+h+1, x+w], na[y+h, x+w+1]]),axis=0)
        na[y+1:y+h, x+w]=np.mean(np.array([na[y+1:y+h, x+w-1], na[y+1:y+h, x+w+1]]),axis=0) 
        na[y, x+w]=np.mean(np.array([na[y-1, x+w], na[y, x+w+1]]),axis=0) 
        na[y, x+1:x+w]=np.mean(np.array([na[y-1, x+1:x+w], na[y+1, x+1:x+w]]),axis=0) 
        diff = np.abs(orig[y+1:y+h, x+1:x+w].astype(np.float32) - na[y+1:y+h, x+1:x+w].astype(np.float32))
        squares=np.square(diff)
        sumofsquares = np.sum(squares)
        rootofsquares = sumofsquares**0.5
        loss=rootofsquares/3
        sum+=loss
        print(loss)
    #if(cnt<10): fname=fulldir+'SquareDetectionResults/decrypted_frames/decrypt0'+str(cnt)
    #else: fname=fulldir+'SquareDetectionResults/decrypted_frames/decrypt'+str(cnt)
    #cv2.imwrite(fname+ext, na)
    #cnt+=1
    sum=sum/3
    return na, sum

In [7]:
cam = cv2.VideoCapture(fulldir+name+"/results/encryptedvideo.avi")
fps = cam.get(cv2.CAP_PROP_FPS)
print(fps)
height = int(cam.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(cam.get(cv2.CAP_PROP_FRAME_WIDTH))
print(height, width)

30.0
720 1280


In [8]:
import os

files=[]
entries = os.listdir(fulldir+name+'/encrypted_frames/')

for entry in entries:
    if entry[-4:]=='.bmp':
        files.append(entry)

files.sort()

print(files)

print(len(files))

['EncryptedResult000.bmp', 'EncryptedResult001.bmp', 'EncryptedResult002.bmp', 'EncryptedResult003.bmp', 'EncryptedResult004.bmp', 'EncryptedResult005.bmp', 'EncryptedResult006.bmp', 'EncryptedResult007.bmp', 'EncryptedResult008.bmp', 'EncryptedResult009.bmp', 'EncryptedResult010.bmp', 'EncryptedResult011.bmp', 'EncryptedResult012.bmp', 'EncryptedResult013.bmp', 'EncryptedResult014.bmp', 'EncryptedResult015.bmp', 'EncryptedResult016.bmp', 'EncryptedResult017.bmp', 'EncryptedResult018.bmp', 'EncryptedResult019.bmp', 'EncryptedResult020.bmp', 'EncryptedResult021.bmp', 'EncryptedResult022.bmp', 'EncryptedResult023.bmp', 'EncryptedResult024.bmp', 'EncryptedResult025.bmp', 'EncryptedResult026.bmp', 'EncryptedResult027.bmp', 'EncryptedResult028.bmp', 'EncryptedResult029.bmp', 'EncryptedResult030.bmp', 'EncryptedResult031.bmp', 'EncryptedResult032.bmp', 'EncryptedResult033.bmp', 'EncryptedResult034.bmp', 'EncryptedResult035.bmp', 'EncryptedResult036.bmp', 'EncryptedResult037.bmp', 'EncryptedR

In [9]:
video_name = 'decryptedvideo.avi'
print(fulldir+name+"/results/")
os.chdir(fulldir+name+"/results/")
video = cv2.VideoWriter(video_name, 0, fps, (width, height))
totalloss=0
loss=0
i=0
for i in range(len(files)):
    if i<10:
        Img= cv2.imread(fulldir+name+"/frames/00"+str(i)+".png")
    elif i<100:
        Img= cv2.imread(fulldir+name+"/frames/0"+str(i)+".png")
    else:
        Img= cv2.imread(fulldir+name+"/frames/"+str(i)+".png")
    temploss=0
    frameout, temploss=DecryptionResult(i, Img)
    totalloss+=temploss
    if i<10:
        cv2.imwrite(fulldir+name+"/decrypted_frames/00"+str(i)+".png", frameout)
    elif i<100:
        cv2.imwrite(fulldir+name+"/decrypted_frames/0"+str(i)+".png", frameout)
    else:
        cv2.imwrite(fulldir+name+"/decrypted_frames/"+str(i)+".png", frameout)
        #if currentframe<10:
            #cv2.imwrite(fulldir+"SquareDetectionResults/encrypted_frames_bmp/EncryptedResult0"+str(currentframe)+".bmp", frame)
        #else:
            #cv2.imwrite(fulldir+"SquareDetectionResults/encrypted_frames_bmp/EncryptedResult"+str(currentframe)+".bmp", frame)
    i=i+1
    loss=loss+np.mean((frameout-Img)**2)
    print("frame: ", i)
    video.write(frameout)

/Users/mahitha0x01/Documents/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/results/
(720, 1280, 3)
[[65, 200, 50, 88], [1020, 274, 30, 37], [939, 291, 22, 28]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
3558.693268290733
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
1823.3908583247373
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
1390.7185193273297
frame:  1
(720, 1280, 3)
[[66, 198, 50, 87], [1023, 270, 34, 39], [942, 291, 21, 27]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
3595.813120839291
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
2045.928096922709
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
1266.1544139637945
frame:  2
(720, 1280, 3)
[[66, 199, 50, 86], [1029, 270, 33, 39], [946, 290, 23, 29]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
3504.938230554142
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl

/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
3524.0808415497195
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
2011.9076630015713
frame:  27
(720, 1280, 3)
[[1155, 255, 53, 72], [994, 283, 35, 39]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
3493.384477997106
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
2066.875688354554
frame:  28
(720, 1280, 3)
[[1163, 250, 58, 70], [998, 279, 34, 41]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
3631.601789232343
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
2088.6395359446565
frame:  29
(720, 1280, 3)
[[1175, 242, 58, 73], [1121, 250, 29, 37], [1000, 276, 36, 42]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
3684.6463421428475
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
1917.654643916191
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
2191.5026808105895
fram

frame:  77
(720, 1280, 3)
[[737, 279, 25, 29], [516, 293, 25, 29]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
1521.5946313727
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
1501.2303842737354
frame:  78
(720, 1280, 3)
[[740, 278, 27, 31], [509, 292, 27, 31], [865, 301, 23, 28]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
1678.886403410176
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
1723.8300380257908
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
1432.6266474944234
frame:  79
(720, 1280, 3)
[[745, 279, 24, 29], [506, 292, 27, 30]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
1424.3045866511684
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
1670.9496966961299
frame:  80
(720, 1280, 3)
[[746, 278, 27, 31], [504, 292, 27, 30]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
1658.8419655490595
/svnit/Assignments/FYP/Secur

[[834, 162, 26, 37], [826, 249, 38, 51], [674, 269, 22, 32], [461, 282, 38, 46], [740, 301, 19, 27], [936, 308, 29, 32]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
1608.138882891234
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
2615.8336678347455
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
1537.4770097648795
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
2420.599650224437
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
1318.9155056754437
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
1694.5233351398065
frame:  105
(720, 1280, 3)
[[834, 163, 25, 35], [831, 249, 40, 52], [460, 282, 39, 46], [742, 301, 18, 26], [939, 309, 27, 32]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
1546.7672381094992
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
2683.4709695549986
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decry

frame:  123
(720, 1280, 3)
[[965, 205, 72, 88], [404, 261, 59, 73], [679, 261, 25, 35], [753, 296, 25, 30]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
4847.711212520812
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
3890.5522030226566
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
1766.843135840493
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
1493.5175481616097
frame:  124
(720, 1280, 3)
[[978, 202, 71, 90], [676, 262, 27, 39], [401, 263, 59, 74], [754, 296, 26, 31]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
4807.359913022254
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
1963.4134675214093
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
3908.5265197457156
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
1619.170912672422
frame:  125
(720, 1280, 3)
[[983, 203, 79, 94], [676, 261, 27, 40], [395, 264, 64, 77], [756, 297, 2

...

frame:  513
(720, 1280, 3)
[[579, 247, 31, 36], [1030, 282, 28, 28], [785, 288, 23, 26]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
2036.0956100668097
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
1481.4785407378222
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
1440.482018091321
frame:  514
(720, 1280, 3)
[[577, 245, 32, 38], [1034, 285, 28, 26], [786, 592, 30, 33]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
2138.734101181247
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
1471.7352569897437
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
1720.4943733964635
frame:  515
(720, 1280, 3)
[[577, 246, 32, 35], [1037, 286, 28, 25]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
1995.6992648080911
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
1420.4188740571487
frame:  516
(720, 1280, 3)
[[575, 245, 30, 37]]
/svnit/Assignments/

/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
3840.3823883682326
frame:  555
(720, 1280, 3)
[[411, 179, 68, 79]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
4218.518960751773
frame:  556
(720, 1280, 3)
[[401, 170, 69, 80]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
4292.9134370236
frame:  557
(720, 1280, 3)
[[389, 156, 75, 83]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
4479.092765281826
frame:  558
(720, 1280, 3)
[[376, 137, 76, 87]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
4674.117100230445
frame:  559
(720, 1280, 3)
[[363, 121, 77, 89]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
4695.681041600296
frame:  560
(720, 1280, 3)
[[346, 105, 82, 93], [1217, 281, 26, 35]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
4989.064307730125
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
1612.5849503893498
frame:  561

(720, 1280, 3)
[[373, 253, 39, 48], [439, 266, 29, 42], [1106, 280, 22, 32], [1141, 302, 30, 39]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
2584.0005159958205
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
2103.558255274461
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
1527.9728037137602
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
1802.6211162884144
frame:  581
(720, 1280, 3)
[[372, 252, 38, 46], [438, 265, 29, 45], [941, 271, 26, 31], [1144, 304, 33, 38]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
2466.3630443766115
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
2098.5438337624073
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
1541.4165202465194
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
1885.5890326367514
frame:  582
(720, 1280, 3)
[[368, 247, 39, 53], [942, 271, 26, 37], [1149, 303, 32, 40]]
/svnit/Assignment

(720, 1280, 3)
[[233, 210, 53, 73], [331, 241, 29, 40], [400, 258, 34, 45], [436, 267, 28, 34], [952, 269, 33, 37], [1110, 288, 22, 28]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
3693.026100338011
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
1971.235483987982
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
2264.2624504337928
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
1844.4677642434779
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
1900.779956871506
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
1354.155374311891
frame:  600
(720, 1280, 3)
[[221, 204, 54, 80], [328, 242, 30, 43], [396, 258, 33, 49], [956, 269, 31, 38]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
3943.0911731787287
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
2086.389225432302
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
2351.

frame:  613
(720, 1280, 3)
[[109, 159, 80, 100], [319, 222, 46, 65], [394, 247, 29, 36], [974, 259, 32, 42], [1226, 261, 37, 40], [1156, 277, 25, 27]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
5243.503832786283
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
3247.363922390659
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
1935.1640757310477
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
2081.8792792410736
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
1978.652740292411
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
1569.1978134632288
frame:  614
(720, 1280, 3)
[[97, 153, 85, 101], [313, 222, 47, 68], [393, 247, 29, 35], [973, 257, 34, 43], [1235, 260, 34, 45], [1158, 277, 24, 28]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
5402.942572750931
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
3350.6378828847232
/svnit/Assignme

In [10]:
video.release() 
print(totalloss/len(files))
print(loss/len(files))

2206.75369472416
0.8398700444983821
